In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import openpyxl,re,time,os

In [54]:
outfile_file

'../data\\data.xlsx'

In [52]:
# Initiate browser driver and open page of interest
geckodriver_path = 'C:/Users/hp/Documents/DAO/webdriver/geckodriver.exe'  # Update with the correct file name and extension
directory_path = '../data'
outfile_file = os.path.join(directory_path,'data.xlsx')

# Check/create output directory
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

browser = webdriver.Firefox(executable_path=geckodriver_path)
browser.get("https://snapshot.org/#/uniswap")

In [3]:
# Infinite scroll till the end of the page
first_run = 1
while True:
    # Execute JavaScript to scroll to the bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for some time for the new content to load
    # You can adjust the sleep duration based on the website and your internet speed
    time.sleep(5)  # Wait for 2 seconds

    # Check if the bottom of the page is reached
    current_height = browser.execute_script("return document.body.scrollHeight")
    if first_run ==1:
        prev_height = 0
        first_run =0
    if current_height == prev_height:
        break  # Reached the end of the page, exit the loop
    else:
        print(f"Current Height - {current_height}\t Prev Height - {prev_height}")
        prev_height = current_height
        print('Moving down')

print("Done scrolling", "\n","*"*15)

Current Height - 4640	 Prev Height - 0
Moving down
Current Height - 6770	 Prev Height - 4640
Moving down
Current Height - 8901	 Prev Height - 6770
Moving down
Current Height - 11024	 Prev Height - 8901
Moving down
Current Height - 13091	 Prev Height - 11024
Moving down
Current Height - 15222	 Prev Height - 13091
Moving down
Current Height - 17384	 Prev Height - 15222
Moving down
Current Height - 19503	 Prev Height - 17384
Moving down
Current Height - 21634	 Prev Height - 19503
Moving down
Current Height - 23657	 Prev Height - 21634
Moving down
Current Height - 25820	 Prev Height - 23657
Moving down
Current Height - 27906	 Prev Height - 25820
Moving down
Current Height - 30069	 Prev Height - 27906
Moving down
Current Height - 32132	 Prev Height - 30069
Moving down
Current Height - 33153	 Prev Height - 32132
Moving down
Done scrolling 
 ***************


In [40]:
# Get the HTML source of the page and parse using bs4
html_source = browser.page_source

soup = BeautifulSoup(html_source, 'html.parser')

In [57]:
# Open Workbook object and create titles
workbook = openpyxl.load_workbook(outfile_file)
sheet = workbook.create_sheet('Snapshot Summary')
sheet.cell(row=1, column=1).value = 'Proposer'
sheet.cell(row=1, column=2).value = 'Title'
sheet.cell(row=1, column=3).value = 'Excerpt Description'
sheet.cell(row=1, column=4).value = 'Status'
sheet.cell(row=1, column=5).value = 'Ending Date'
sheet.cell(row=1, column=6).value = 'Link'
sheet.cell(row=1, column=7).value = 'Option 1'
sheet.cell(row=1, column=8).value = 'Option 2'
sheet.cell(row=1, column=9).value = 'Option 3'

workbook.save(outfile_file)

In [58]:
# Perform data extraction using BeautifulSoup methods
data = soup.find_all('div', class_='border-y border-skin-border bg-skin-block-bg text-base md:rounded-xl md:border transition-colors')

In [59]:
len(data)

93

In [64]:
row = 2
for discussion in data:
    proposer = discussion.find('span', class_='w-full cursor-pointer truncate text-skin-link').text
    title = discussion.find('h3', class_='inline pr-2').text
    try:
        excerpt = discussion.find('p', class_='mb-2 line-clamp-2 break-words text-md').text[:50]
    except:
        excerpt = ""
    status = discussion.find('span', class_='bg-violet-600 State text-white').text
    ending_date = discussion.find('span', class_='cursor-help text-sm').text
    link = 'https://snapshot.org/'+discussion.find('a',class_="")['href']
    options = discussion.find_all('div', class_='relative mt-1 w-full')
    count = 0
    for number,option in enumerate(options):
        try:
            option_text = option.find('div', class_='absolute ml-3 flex items-center leading-[43px] text-skin-link').text
            option_name = ' '.join(option_text.split()[:-2])
            option_coin = option.find('span',class_='ml-1 text-skin-text').text
            option_percentage = option.find('div', class_='absolute right-0 mr-3 leading-[40px] text-skin-link').text
            option_details = (option_name,option_coin,option_percentage)
            sheet.cell(row=row, column=7+count).value = f"{option_name}, {option_coin}, {option_percentage}"
            print(option_details)
#             print(index, count, option_details)
            count+=1
            
        except:
            print('Error here')
            pass

    sheet.cell(row=row, column=1).value = proposer
    sheet.cell(row=row, column=2).value = title
    sheet.cell(row=row, column=3).value = excerpt
    sheet.cell(row=row, column=4).value = status
    sheet.cell(row=row, column=5).value = ending_date
    sheet.cell(row=row, column=6).value = link
    workbook.save(outfile_file)
    
    row+=1
    
    

('Yes', '49M UNI', '100%')
('No', '95 UNI', '0%')
('Abstain', '1.5K UNI', '0%')
('Yes', '29M UNI', '100%')
('No', '115 UNI', '0%')
('Abstain', '1.1K UNI', '0%')
Error here
('USDC', '9M UNI', '54.67%')
Error here
Error here
('Polygon', '22M UNI', '99.96%')
('Optimism', '5.7K UNI', '0.03%')
('Arbitrum', '2.9K UNI', '0.01%')
Error here
Error here
Error here
('No Fee', '18M UNI', '45.32%')
('Yes', '26M UNI', '100%')
('No', '169 UNI', '0%')
('Abstain', '717 UNI', '0%')
('Yes - create new subdomain', '29M UNI', '100%')
('No - do not create new subdomain', '786 UNI', '0%')
('Abstain', '225 UNI', '0%')
('Yes', '20M UNI', '100%')
('No', '200 UNI', '0%')
('Yes', '20M UNI', '100%')
('No', '508 UNI', '0%')
('Yes', '20M UNI', '100%')
('No', '918 UNI', '0%')
('Yes', '26M UNI', '91.26%')
('No', '2.5M UNI', '8.74%')
('Abstain', '94 UNI', '0%')
('Doo Wan Nam (StableLab)', '16M UNI', '69.24%')
Error here
Error here
Error here
Error here
Error here
('Yes', '1M UNI', '99.88%')
('No (make no change)', '1.3

In [63]:
row = 2
for index,discussion in enumerate(data):
    options = discussion.find_all('div', class_='relative mt-1 w-full')
    count = 0
    for number,option in enumerate(options):
        try:
            option_text = option.find('div', class_='absolute ml-3 flex items-center leading-[43px] text-skin-link').text
            option_name = ' '.join(option_text.split()[:-2])
            option_coin = option.find('span',class_='ml-1 text-skin-text').text
            option_percentage = option.find('div', class_='absolute right-0 mr-3 leading-[40px] text-skin-link').text
            option_details = (option_name,option_coin,option_percentage)
            print(index, count, option_details)
            count+=1
            
        except:
            print('Error here')
            pass

        
    
#     print(index,'https://snapshot.org/'+discussion.find('a',class_="")['href'])
    
      

0 0 ('Yes', '49M UNI', '100%')
0 1 ('No', '95 UNI', '0%')
0 2 ('Abstain', '1.5K UNI', '0%')
1 0 ('Yes', '29M UNI', '100%')
1 1 ('No', '115 UNI', '0%')
1 2 ('Abstain', '1.1K UNI', '0%')
Error here
2 0 ('USDC', '9M UNI', '54.67%')
Error here
Error here
3 0 ('Polygon', '22M UNI', '99.96%')
3 1 ('Optimism', '5.7K UNI', '0.03%')
3 2 ('Arbitrum', '2.9K UNI', '0.01%')
Error here
Error here
Error here
4 0 ('No Fee', '18M UNI', '45.32%')
5 0 ('Yes', '26M UNI', '100%')
5 1 ('No', '169 UNI', '0%')
5 2 ('Abstain', '717 UNI', '0%')
6 0 ('Yes - create new subdomain', '29M UNI', '100%')
6 1 ('No - do not create new subdomain', '786 UNI', '0%')
6 2 ('Abstain', '225 UNI', '0%')
7 0 ('Yes', '20M UNI', '100%')
7 1 ('No', '200 UNI', '0%')
8 0 ('Yes', '20M UNI', '100%')
8 1 ('No', '508 UNI', '0%')
9 0 ('Yes', '20M UNI', '100%')
9 1 ('No', '918 UNI', '0%')
10 0 ('Yes', '26M UNI', '91.26%')
10 1 ('No', '2.5M UNI', '8.74%')
10 2 ('Abstain', '94 UNI', '0%')
11 0 ('Doo Wan Nam (StableLab)', '16M UNI', '69.24%')

In [ ]:
#     # Save data points in specific cells
#     sheet.cell(row=row, column=1).value = title
#     sheet.cell(row=row, column=2).value = description
#     sheet.cell(row=row, column=3).value = reply
#     sheet.cell(row=row, column=4).value = views
#     sheet.cell(row=row, column=5).value = first_post_date
#     sheet.cell(row=row, column=6).value = posted_date
#     sheet.cell(row=row, column=7).value = link
#     row+=1
    
#     sheet.parent.save(outfile_file)

# browser.quit()